In [1]:
import pandas as pd
import numpy as np
import joblib

from scipy.spatial.distance import cosine, jensenshannon
from sklearn.preprocessing import normalize

from plotnine import ggplot, aes, geoms
from tqdm import tqdm

In [2]:
with open('Results/Official_TopicModel_80k.pkl','rb') as File:
    models = joblib.load(File)
    
all_df = pd.read_csv('Results/All_speeches_labelled.csv')

In [3]:
def select_data(year):
    """
    convenience function for extracting X,Y data and dataframe
    """
    sub_df = all_df.loc[all_df.year == year]
    model = [mod for mod in models['window_models'] if mod['year'] == year][0]    
    W = normalize(model['W'])
    
    return W,sub_df[['party','unique_ID']].reset_index()

In [4]:
def get_avg_dist_partisan(df,W,speaker):
    speaker_df_index = df.loc[df.unique_ID == speaker].index
    speaker_W = W[speaker_df_index,:]
    return np.mean(speaker_W,0)


def run_pairwise_cos(mat):
    cos_mat = np.zeros([mat.shape[0],mat.shape[0]])
    for i,dist in enumerate(mat):
        for ii, dist2 in enumerate(mat):
            if i != ii:
                cos_mat[i,ii] = 1 - jensenshannon(dist,dist2)

    upper_tri = cos_mat[np.triu_indices_from(cos_mat,1)]
    mean = np.mean(upper_tri)
    std = np.std(upper_tri)
    return mean,std

In [5]:
    
def Run_year(year):
    W,df = select_data(year)

    row = {}

    speakers = df.groupby('unique_ID').first().reset_index()

    avg_dists = []
    for speaker in speakers.iterrows():
        speaker = speaker[1]
        avg_dist = get_avg_dist_partisan(df,W,speaker.unique_ID)
        avg_dists.append({"unique_ID":speaker.unique_ID,'party':speaker.party,'dist':avg_dist})

    year_df = pd.DataFrame(avg_dists)

    D_matrix = np.array([x for x in year_df.loc[year_df.party == 'D','dist'].values])
    R_matrix = np.array([x for x in year_df.loc[year_df.party == 'R','dist'].values])

    row['D_mean'],row['D_std'] = run_pairwise_cos(D_matrix)
    row['R_mean'],row['R_std'] = run_pairwise_cos(R_matrix)

    row['year'] = year
    return row

In [6]:
consolidation = []
for year in tqdm(range(1983,2017)):
    consolidation.append(Run_year(year))

100%|██████████| 34/34 [01:18<00:00,  2.31s/it]


In [7]:
all_df = pd.DataFrame(consolidation)

In [8]:
all_df.to_csv('Results/consolidation_new.csv')